<a href="https://colab.research.google.com/github/chrishare/colab_deeplearning/blob/master/pytorch-huggingface-whipser-youtube-asr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pwd
!ls -ltr

/content
total 4
drwxr-xr-x 1 root root 4096 Mar 30 13:53 sample_data


In [2]:
!pip install git+https://github.com/ytdl-org/youtube-dl.git@master#egg=youtube_dl
!pip install yt-dlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ytdl-org/youtube-dl.git (to revision master) to /tmp/pip-install-z61snptt/youtube-dl_b361cec5c1934e2f921866657dac37b1
  Running command git clone --filter=blob:none --quiet https://github.com/ytdl-org/youtube-dl.git /tmp/pip-install-z61snptt/youtube-dl_b361cec5c1934e2f921866657dac37b1
  Resolved https://github.com/ytdl-org/youtube-dl.git to commit 6fece0a96b3cd8677f5c1185a57c6e21403fcb44
  Preparing metadata (setup.py) ... done
  Created wheel for youtube_dl: filename=youtube_dl-2021.12.17-py2.py3-none-any.whl size=1920289 sha256=5f7d3a6bfea9e54a2194998569f8c57ae7b8dd51603fdb0fb02e044d35e31351
  Stored in directory: /tmp/pip-ephem-wheel-cache-2jvsy_1e/wheels/e5/86/de/31b768b5113d882def26b85ab4f11761f20229c8493751653f
Successfully built youtube_dl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━

In [3]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [4]:
!yt-dlp https://www.youtube.com/watch?v=xmYekD6-PZ8 --extract-audio --audio-format "mp3" -o sample.mp3

[youtube] Extracting URL: https://www.youtube.com/watch?v=xmYekD6-PZ8
[youtube] xmYekD6-PZ8: Downloading webpage
[youtube] xmYekD6-PZ8: Downloading android player API JSON
[info] xmYekD6-PZ8: Downloading 1 format(s): 251
[dashsegments] Total fragments: 6
[download] Destination: sample.webm
[download] 100% of   50.52MiB in 00:00:01 at 32.63MiB/s
[ExtractAudio] Destination: sample.mp3
Deleting original file sample.webm (pass -k to keep)


In [5]:
!ls -ltr

total 44228
-rw-r--r-- 1 root root 45284709 Mar  2  2021 sample.mp3
drwxr-xr-x 1 root root     4096 Mar 30 13:53 sample_data


In [6]:
!pip install git+https://github.com/huggingface/transformers.git@main
!pip install git+https://github.com/huggingface/datasets.git@main

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-vpbjednw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-vpbjednw
  Resolved https://github.com/huggingface/transformers.git to commit c612628045822f909020f7eb6784c79700813eda
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.7 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.28.0.dev0-py3-none-any.whl size=6858427 sha256=0014f73c83920c6ef4a7fd57dc1da3872913212031670ec06db8cb556750a60b
  Stored in directory: /tmp/pip-ephem-wheel-cache-zo053_aq/wheels/b1/e5/2d/f924f6ef6b8b

In [9]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-medium")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium").to("cuda")
model.config.forced_decoder_ids = None


In [12]:
# Importing the library
import psutil
 
# Getting % usage of virtual_memory ( 3rd field)
print('RAM memory % used:', psutil.virtual_memory()[2])
# Getting usage of virtual_memory in GB ( 4th field)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

RAM memory % used: 29.8
RAM Used (GB): 3.725406208


In [11]:
# Load some audio
!pip install whisper-openai
import whisper
audio = whisper.load_audio("./sample.mp3")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.8 MB/s eta 0:00:00


In [13]:
from transformers import pipeline

In [14]:
print(audio)

[0. 0. 0. ... 0. 0. 0.]


In [16]:
#i = processor(audio,return_tensors="pt").input_features.to("cuda:0")
i = processor(audio,return_tensors="pt").input_features
pipe = pipeline(
  "automatic-speech-recognition",
  model=model,
  tokenizer=processor.tokenizer,
  feature_extractor=processor.feature_extractor,
  chunk_length_s=30,
  device="cuda:0",
)


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [23]:
r = pipe("sample.mp3", generate_kwargs  = {"task":"transcribe", "language":"english"})

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [26]:
print(r)

{'text': " All right, good morning. Good morning. Wow, that was pretty good actually. Thank you. I'm Jay Hartzell. I'm the dean. It's my pleasure to see everybody here today and thanks for coming out on a Friday. This is one of the craziest weeks on campus all year. We started building into this week and thinking, into the year thinking what's gonna be this week and it turns out people don't mind being around when there's a USC coming on Saturday. So it's a really great week for us to bring back some superstars and Bill Gurley's definitely one of those superstars. So thanks Bill for coming back to campus. We've been working on trying to get him back in the right time and the right venue and we've had this conversation around what he wanted to share with our MBA students and so this is a great chance to get him back. So thank you all for coming out. Thanks to Tina Mabely and her team for arm twisting and making sure that you all knew about this. Also thanks to our faculty. So Jim Nolan'